# MNIST-cGAN

Generate handwritten digits by training a cGAN on the MNIST dataset.

This project is licensed under the MIT License.

## Introduction

This project assumes a basic understanding of Generative Adversarial Networks (GANs). It builds on a basic MNIST-GAN. <br>
For a comprehensive introduction to GANs and an example implementation in pytorch, check out [MNIST-GAN](https://github.com/umcconnell/mnist-gan)

Conditional GANS (cGANs) are a specific type of GANs that can produce a user-specified class of images instead of only randomingly reproducing the input training set, as it is the case with plain GANs. For example, a cGAN can produce a specified handwritten digit instead of outputting any handwritten digit.

To achieve this, a cGAN receives additional labels as inputs, specifying which class of images to reproduce.

### Training

Like with a plain GAN, training is done in two steps.

#### Step 1: Training the Discriminator

In the first step, the discriminator is trained. The discriminator is fed some real samples from the training set, labelled as real (usually with a 1), and their corresponding digit label, such as `8`. Additionally, some fake samples generated by the discriminator, labelled as fake samples (usually with a 0), and their corresponding fake digit label, such as `3`, are fed to the discriminator. Then, standard backpropagation is applied onto the discriminator only.

![Step 1: Training the discriminator](../static/cgan/figure1.png)

#### Step 2: Training the Generator

Next, the generator is trained. First, the generator generates a batch of fake images with random noise and fake digit labels. The fake images and their corresponding digit labels are fed to the discriminator with real labels (usually a `1`). The error is then backpropagated to the generator, but only the generator weights are updated.

![Step 2: Training the Generator](../static/cgan/figure2.png)

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
BATCH_SIZE = 200
HALF_BATCH = int(BATCH_SIZE/2)
EPOCHS = 20
NUM_CLASSES = 10

# GAN configurations
LATENT_DIM = 100
REAL_LABEL = 1
FAKE_LABEL = 0

## Loading MNIST Dataset

We'll be using the MNIST dataset to train our GAN. It contains images of handwritten digits. Loading MNIST is trivial using torchvision.

Before we can use the images to train the network, it's a best practice to normalize the images. The images are black-and-white, represented by values from [0, 1]. The transformation will bring the values in a range of [-1, 1]:

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

In [ ]:
dataset = torchvision.datasets.MNIST(
    root='./data', download=True, transform=transform
)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=HALF_BATCH, shuffle=True, num_workers=2
)

## Visualizing

Let's visualize our training set before actually using it:

In [ ]:
def show_img(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(npimg[:, :], cmap='gray_r')
    plt.show()

In [ ]:
dataiter = iter(dataloader)
imgs, labels = next(dataiter)

show_img(imgs[0].squeeze())
print("Label %i" % labels[0].item())

## Latent space

The generator creates new images by upsampling a random seed, the so-called latent space.

We'll create a helper function that creates a minibatch of latent spaces:

In [ ]:
def generate_latent_points(latent_dim, n_samples):
    return torch.rand(n_samples, latent_dim)

In [ ]:
# Show random latent_space / seed
img = generate_latent_points(28*28, 1).view(28, -1)
show_img(img)

## Model

The `Model` class will be used as a base for the `Generator` and `Discriminator` class. It mainly offers the predict method, to run the model without accumulating gradients, and the train_on method, used to train a model on a batch and backpropagate.

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
    def forward(self, x):
        return self.model(x)
    
    def predict(self, *x):
        with torch.no_grad():
            return self.forward(*x).detach()
        
    def train_on(self, x, y, criterion, optimizer):
        output = self.forward(*x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        return loss

We'll also define a custom embed layer, that turns the MNIST digit labels, such as `6`, into one-hot vectors with a `1` at the digit label index. For the digit label `6` this would be `[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]`. Check out the [pytorch docs](https://pytorch.org/docs/stable/nn.functional.html#one-hot) for more information.

In [ ]:
class Embed(nn.Module):
    def __init__(self):
        super(Embed, self).__init__()
        
    def forward(self, label):
        # you can also try:
        # return F.embedding(NUM_CLASSES, NUM_CLASSES)
        return F.one_hot(label, NUM_CLASSES).float()

## Generator

First we'll define a helper class to reshape tensors. This will be used as a layer in our network and reshape input tensors to a desired size:

In [ ]:
class Reshape(nn.Module):
    def __init__(self, shape):
        super(Reshape, self).__init__()
        self.shape = shape
    
    def forward(self, x):
        return x.view(*self.shape)

The Generator takes a latent space and a digit label as input and upsamples this seed with the embedded digit label to the desired image of the digit:

In [ ]:
class Generator(Model):
    def __init__(self):
        super(Generator, self).__init__()
                
        self.image = nn.Sequential(
            nn.Linear(LATENT_DIM, 128*7*7),
            nn.BatchNorm1d(128*7*7),
            nn.LeakyReLU(0.2),
            Reshape((-1, 128, 7, 7)),
        )
        
        self.label = nn.Sequential(
            Embed(),
            nn.Linear(10, 7*7),
            Reshape((-1, 1, 7, 7))
        )
        
        self.main = nn.Sequential(
            # Upsample to 14x14
            nn.ConvTranspose2d(129, 128, 4, stride=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            
            # Upsample to 28x28
            nn.ConvTranspose2d(128, 128, 4, stride=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(128, 1, 7),
            nn.Tanh()
        )
        
    def forward(self, img, label):
        img = self.image(img)
        label = self.label(label)
        
        x = torch.cat((img, label), dim=1)
        
        return self.main(x)

### Generator Test

Let's test the generator just to check that it works:

In [ ]:
generator_example = Generator()
generator_example.train(False) # Set the generator to evaluation mode
latent_points_example = generate_latent_points(LATENT_DIM, 1)
digit_label_example = torch.tensor(8)

generated_image_example = generator_example.predict(latent_points_example, digit_label_example)
show_img(generated_image_example.squeeze())

## Discriminator


The discriminator takes in an image with a corresponding digit label and assesses, whether the image is real or fake:

In [ ]:
class Discriminator(Model):
    def __init__(self):
        super(Discriminator, self).__init__()
                
        self.label = nn.Sequential(
            Embed(),
            nn.Linear(10, 28*28),
            Reshape((-1, 1, 28, 28))
        )
        
        self.main = nn.Sequential(
            nn.Conv2d(2, 128, 3, stride=2),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(128, 128, 3, stride=2),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(128, 128, 3, stride=2),
            nn.LeakyReLU(0.2),
            
            nn.Flatten(),
            nn.Dropout(0.4),
                        
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, img, label):
        label = self.label(label)
        
        x = torch.cat((img, label), dim=1)
        
        return self.main(x)

### Discriminator Test

Let's now test discriminator just to check that it works:

In [ ]:
discriminator_example = Discriminator()

prediction_example = discriminator_example.predict(generated_image_example, digit_label_example)
print('Discriminator Prediction: %f' % prediction_example.item())

## Training

First we'll try setting up pytorch to use a CUDA-capable GPU. If no GPU is detected, the GAN will be trained on CPU:

In [ ]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device "%s" for training' % dev)

Next we'll create the networks and move them to our selected device:

In [ ]:
generator = Generator().to(dev)
discriminator = Discriminator().to(dev)

We'll also define the optimizers, used to train the networks, and our loss function:

In [ ]:
generator_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
criterion = nn.BCELoss()

Now let's finally train our cGAN:

In [ ]:
for epoch in range(EPOCHS):
    for i, (imgs, digit_labels) in enumerate(dataloader, 0):
        # ===========================
        # STEP 1: Train Discriminator
        # ===========================        
        imgs, digit_labels = imgs.to(dev), digit_labels.to(dev)
        
        discriminator_real = torch.zeros(HALF_BATCH).fill_(REAL_LABEL)
        discriminator_fake = torch.zeros(HALF_BATCH).fill_(FAKE_LABEL)
        
        # Fake training data
        fake_digit_labels = torch.randint(NUM_CLASSES, (HALF_BATCH,)).to(dev)
        
        z = generate_latent_points(LATENT_DIM, HALF_BATCH).to(dev)
        fake_imgs = generator.predict(z, fake_digit_labels)

        discriminator_optimizer.zero_grad()

        # Combine real and fake half-batches to one full batch 
        images_all = torch.cat((imgs, fake_imgs))
        labels_all = torch.cat((digit_labels, fake_digit_labels)).to(dev)
        discriminator_all = torch.cat((discriminator_real, discriminator_fake)).unsqueeze(1).to(dev)
        
        discriminator_loss = discriminator.train_on(
            (images_all, labels_all), discriminator_all, criterion, discriminator_optimizer
        )
        
        # =======================
        # STEP 2: Train Generator
        # =======================
        generator_optimizer.zero_grad()

        fake_batch = generate_latent_points(LATENT_DIM, BATCH_SIZE).to(dev)
        fake_digit_labels = torch.randint(NUM_CLASSES, (BATCH_SIZE,)).to(dev)
        discriminator_fake = torch.zeros(BATCH_SIZE).fill_(REAL_LABEL).unsqueeze(1).to(dev)

        generator_imgs = generator.forward(fake_batch, fake_digit_labels)
        generator_loss = criterion(discriminator(generator_imgs, fake_digit_labels), discriminator_fake)
        generator_loss.backward()
        generator_optimizer.step()

        # ===============================
        # STEP 3: Logging and Visualizing
        # ===============================
        if i % 25 == 0:
            print('Epoch: %.2i    Batch Number: %.3i / %.3i    Generator Loss: %.9f    Discriminator Loss: %.9f' %
                 (epoch, i, len(dataloader), generator_loss.item(), discriminator_loss.item()))
            
            generator.eval()
            
            z = generate_latent_points(LATENT_DIM, 30).to(dev)
            # Generate digit labels 1 through 9 to arrange same digits in columns
            y = torch.tensor([x % 10 for x in range(30)]).to(dev)
            fake_imgs = generator.predict(z, y).to("cpu").detach()
            
            fig = plt.figure()
            for a in range(30):
                ax = fig.add_subplot(3, 10, a + 1)
                ax.axes.get_xaxis().set_ticks([])
                ax.axes.get_yaxis().set_ticks([])
                
                plt.imshow(fake_imgs[a, 0, :, :], cmap='gray_r')
            
            plt.savefig('figs/plot  epoch ' + '%02d' % epoch + '  batch ' + '%05d' % i + '.png', dpi=600)
            plt.close()
            
            generator.train()